In [1]:
import pandas as pd #principal herramienta de gestion de datasets como dataframe
import numpy as np #para trabajar con arrays y algunas cuestiones estadisticas
from sklearn import preprocessing #procesamiento de detasets 
from sklearn.metrics import mean_squared_error #herramienta estadistica para medir errores
import seaborn as sns #herramienta grafica
sns.set()
from sklearn.model_selection import train_test_split #para separar el dataset en entrenamiento y testeo
from sklearn.tree import DecisionTreeClassifier #modelo de arbol
from sklearn.metrics import confusion_matrix #para crear la matriz de confusión
from sklearn.metrics import ConfusionMatrixDisplay #para graficar la matriz de confusión
from sklearn.model_selection import GridSearchCV #para optimización de hiperparámetro
import joblib #airve para bajar el modelo entrenado y guardarlo como un achivo
from sklearn import tree #modelo de arbol
import matplotlib.pyplot #herramienta grafica
import matplotlib.pyplot as plt #herramienta grafica
from sklearn.model_selection import cross_validate #para validacion cruzada 
from sklearn.metrics import accuracy_score #metrica accuracy
from sklearn.metrics import precision_score #metrica precisión
from sklearn.metrics import recall_score #metrica recall
from sklearn.metrics import f1_score #metrica f1 score
import warnings #evita que se llene de warnings el script
warnings.filterwarnings("ignore", category=FutureWarning) #evita que se llene de warnings el script
from sklearn.preprocessing import OrdinalEncoder # preprocesamiento de columna


In [2]:
#clono el repositorio

In [3]:
!git clone https://github.com/soyHenry/Datathon.git

Cloning into 'Datathon'...
remote: Enumerating objects: 83, done.
remote: Total 83 (delta 0), reused 0 (delta 0), pack-reused 83
Unpacking objects: 100% (83/83), done.


In [4]:
#ingesto los datos en dataframe de pandas

In [5]:
df_train=pd.read_csv('Datathon/hospitalizaciones_train.csv')

In [6]:
df1=df_train.copy() #realizo una copia para trabajar con back up de los datos crudos 

In [7]:
#creo una funcion para adaptar la columna label a categórica

In [8]:
def transf_label(df):
    lista_dias=df['Stay (in days)']
    lista_estancia=[]
    for i in lista_dias:
        if i <=8: 
            i=0
        elif i>8:
            i=1
        lista_estancia.append(i)
    array_estancia=np.array(lista_estancia)
    array_estancia.reshape(1,-1)
    array_estancia.shape
    df['estancia']=array_estancia
    df.drop(columns=['Stay (in days)'], inplace=True)
    return df

In [9]:
#aplico la función

In [10]:
transf_label(df1)

,Available Extra Rooms in Hospital,Department,Ward_Facility_Code,doctor_name,staff_available,patientid,Age,gender,Type of Admission,Severity of Illness,health_conditions,Visitors with Patient,Insurance,Admission_Deposit,estancia
0,4,gynecology,D,Dr Sophia,0,33070,41-50,Female,Trauma,Extreme,Diabetes,4,Yes,2966.408696,0
1,4,gynecology,B,Dr Sophia,2,34808,31-40,Female,Trauma,Minor,Heart disease,2,No,3554.835677,1
2,2,gynecology,B,Dr Sophia,8,44577,21-30,Female,Trauma,Extreme,Diabetes,2,Yes,5624.733654,0
3,4,gynecology,D,Dr Olivia,7,3695,31-40,Female,Urgent,Moderate,None,4,No,4814.149231,0
4,2,anesthesia,E,Dr Mark,10,108956,71-80,Male,Trauma,Moderate,Diabetes,2,No,5169.269637,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
409995,1,gynecology,D,Dr Sarah,1,63105,31-40,Female,Trauma,Minor,Asthama,3,Yes,3902.291076,0
409996,5,gynecology,D,Dr Nathan,5,129292,21-30,Female,Trauma,Extreme,Other,4,No,4771.995223,1
409997,4,gynecology,D,Dr Sarah,9,11399,21-30,Female,Emergency,Minor,Asthama,4,Yes,3816.994210,1
409998,10,gynecology,D,Dr Olivia,10,75003,21-30,Female,Trauma,Moderate,Other,2,Yes,3841.577491,1


In [11]:
#Realizo una transormación de las columnas categoricas a numéricas 
#con label encoder en las no ordinales y ordinal encoder en las que si

In [12]:
le = preprocessing.LabelEncoder()
df1['Insurance'] = le.fit_transform(df1['Insurance'])
df1['health_conditions'] = le.fit_transform(df1['health_conditions'])
df1['Type of Admission'] = le.fit_transform(df1['Type of Admission'])
df1['gender'] = le.fit_transform(df1['gender'])
df1['doctor_name'] = le.fit_transform(df1['doctor_name'])
df1['Ward_Facility_Code'] = le.fit_transform(df1['Ward_Facility_Code'])
df1['Department'] = le.fit_transform(df1['Department'])

In [13]:
enc = OrdinalEncoder()
df1['Age'] = enc.fit_transform(df1[['Age']])
df1['Severity of Illness'] = enc.fit_transform(df1[['Severity of Illness']])

In [14]:
df2=df1.copy() #se generan 2 data frame con distintas columnas para aplicar el modelo y analizar diferencias 

In [15]:
df3=df1.copy() #se generan 2 data frame con distintas columnas para aplicar el modelo y analizar diferencias 

In [63]:
df4=df1.copy()

In [16]:
# ACONTINUACIÓN SE PUEDE VER TODOS LOS ARBOLES GENERADOS Y SUS METRICAS CONSIDERANDO DISTINTAS FREATURES 

#_ ESTE PUNTO SE EXPLICA MEJOR EN EL TXT ESTRATEGIA FINAL:
# X1 se hace considerando todas las columnas que INTUITIVAMENTE podrían llegar a impactar 
#X2 son con la elección de las features en base a la correlacion y feature selection de sklearn

y1=df1[['estancia']]
y2=df2[['estancia']]
#TODO _ x=df[['Available Extra Rooms in Hospital','Department','Ward_Facility_Code','doctor_name','staff_available','patientid','Age','gender','Type of Admission','Severity of Illness','health_conditions','Visitors with Patient','Insurance','Admission_Deposit']]
X1=df1[['Available Extra Rooms in Hospital','doctor_name','Department','Severity of Illness','health_conditions','Visitors with Patient','Insurance','Admission_Deposit']]
X2=df2[['Department','Age', 'doctor_name']]


'\nEl valor de la precisión para este modelo es de [0.36356172 0.61923465]\nEl valor de la recall es de [0.17155649 0.81772337]\nEl valor de accuracy es de 0.5736666666666667\nEl f1-score es de:[0.23311251 0.70477027], type[0.23311251 0.70477027]\n'

In [18]:
#COMO PUEDE VERSE ESTO SE RELIZA DOS VECES PORQUE SE ESTAN RELIZANDO 2 ARBOLES CON DISTINTA CANTIDAD DE FEATURES CADA UNO 
#PARA COMPARAR SU EFICACIA EN DETECCION DE FALSOS POSITIVOS, ESTE FUE UN PUNTO FLOJO A SOLUCIONAR DURANTE EL PROCESO DE MODELADO

In [19]:
#realizo la primera partición de los datos, luego se hará una validación cruzada

X2_train, X2_test, y2_train, y2_test = train_test_split(X2, y2, test_size=0.30, random_state=20) 

# datos, dejo un 30% para testear

print(X2_train.shape, y2_train.shape, X2_test.shape, y2_test.shape)

(287000, 3) (287000, 1) (123000, 3) (123000, 1)


In [20]:
#CON EL SIGUIENTE CODIGO SE DESARROLLO UN ENSAYO DEL HIOPERARÁMETRO OPTIMO CON VALIDACIÓN CRUZADA 

In [58]:
pip install autogluon

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 62 kB 668 kB/s 
     |████████████████████████████████| 286 kB 18.1 MB/s 
     |████████████████████████████████| 103 kB 46.6 MB/s 
     |████████████████████████████████| 59 kB 6.7 MB/s 
     |████████████████████████████████| 226 kB 68.4 MB/s 
     |████████████████████████████████| 49 kB 4.3 MB/s 
     |████████████████████████████████| 289 kB 54.3 MB/s 
     |████████████████████████████████| 41 kB 436 kB/s 
     |████████████████████████████████| 802 kB 57.9 MB/s 
     |████████████████████████████████| 1.0 MB 48.9 MB/s 
     |████████████████████████████████| 132 kB 40.0 MB/s 
     |████████████████████████████████| 60.2 MB 88 kB/s 
     |████████████████████████████████| 1.6 MB 43.8 MB/s 
     |████████████████████████████████| 280 kB 37.5 MB/s 
     |████████████████████████████████| 549 kB 61.1 MB/s 
     |████████████████████████████████| 

In [60]:
from autogluon.tabular import TabularPredictor

In [61]:

predictor = TabularPredictor(label='estancia').fit(train_data=df3)

No path specified. Models will be saved in: "AutogluonModels/ag-20221215_233541/"
	Consider setting `time_limit` to ensure training finishes within an expected duration or experiment with a small portion of `train_data` to identify an ideal `presets` and `hyperparameters` configuration.
Beginning AutoGluon training ...
AutoGluon will save models to "AutogluonModels/ag-20221215_233541/"
AutoGluon Version:  0.6.1
Python Version:     3.8.16
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP Fri Aug 26 08:44:51 UTC 2022
Train Data Rows:    410000
Train Data Columns: 14
Label Column: estancia
Preprocessing data ...
AutoGluon infers your prediction problem is: 'binary' (because only two unique label-values observed).
	2 unique label values:  [0, 1]
	If 'binary' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Selected class <--

[1000]	valid_set's binary_error: 0.211951
[2000]	valid_set's binary_error: 0.201707
[3000]	valid_set's binary_error: 0.192439
[4000]	valid_set's binary_error: 0.191707
[5000]	valid_set's binary_error: 0.189512
[6000]	valid_set's binary_error: 0.186098
[7000]	valid_set's binary_error: 0.190976


	0.8144	 = Validation score   (accuracy)
	238.41s	 = Training   runtime
	1.12s	 = Validation runtime
Fitting model: RandomForestGini ...
	0.7844	 = Validation score   (accuracy)
	239.29s	 = Training   runtime
	0.35s	 = Validation runtime
Fitting model: RandomForestEntr ...
	0.7854	 = Validation score   (accuracy)
	277.39s	 = Training   runtime
	0.36s	 = Validation runtime
Fitting model: CatBoost ...
	0.79	 = Validation score   (accuracy)
	396.84s	 = Training   runtime
	0.02s	 = Validation runtime
Fitting model: ExtraTreesGini ...
	0.7346	 = Validation score   (accuracy)
	118.39s	 = Training   runtime
	0.33s	 = Validation runtime
Fitting model: ExtraTreesEntr ...
	0.7351	 = Validation score   (accuracy)
	127.14s	 = Training   runtime
	0.39s	 = Validation runtime
Fitting model: NeuralNetFastAI ...
	0.7632	 = Validation score   (accuracy)
	329.16s	 = Training   runtime
	0.09s	 = Validation runtime
Fitting model: XGBoost ...
	0.7212	 = Validation score   (accuracy)
	3.16s	 = Training   run

In [64]:
pred_probs = predictor.predict_proba(df4)
pred_probs.head(5)

,0,1
0,0.641064,0.358936
1,0.326852,0.673148
2,0.481267,0.518733
3,0.520219,0.479781
4,0.012208,0.987792


In [65]:
results = predictor.fit_summary(show_plot=True)

*** Summary of fit() ***
Estimated performance of each model:
                  model  score_val  pred_time_val    fit_time  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0   WeightedEnsemble_L2   0.814878       1.150928  243.739774                0.014203           2.168311            2       True         14
1              LightGBM   0.814390       1.115447  238.407806                1.115447         238.407806            1       True          4
2              CatBoost   0.790000       0.019389  396.839482                0.019389         396.839482            1       True          7
3         LightGBMLarge   0.785854       0.052196   19.583587                0.052196          19.583587            1       True         13
4      RandomForestEntr   0.785366       0.355169  277.391836                0.355169         277.391836            1       True          6
5      RandomForestGini   0.784390       0.350688  239.290754                0.350688         239.

In [66]:
joblib.dump(predictor,'autogluon_trained')

['autogluon_trained']